Code to mount Drive 


```
from google.colab import drive
drive.mount('/content/drive') 
```




In [ ]:
pip install deep-forest

     |████████████████████████████████| 2.4MB 37.0MB/s 


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from deepforest import CascadeForestClassifier

In [ ]:
pickle_in = open("/content/drive/MyDrive/Ball_bearing_test_0/X.pickle","rb")
X = pickle.load(pickle_in)
pickle_in = open("/content/drive/MyDrive/Ball_bearing_test_0/Y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,shuffle=True,stratify=y)

Training and evaluating accuracy for CNN model

In [ ]:
feature_extractor = Sequential()
feature_extractor.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
feature_extractor.add(BatchNormalization())
feature_extractor.add(Activation('relu'))
feature_extractor.add(MaxPooling2D(pool_size=(2, 2)))

feature_extractor.add(Conv2D(256, (3, 3)))
feature_extractor.add(Activation('relu'))
feature_extractor.add(Dropout(0.5))
feature_extractor.add(MaxPooling2D(pool_size=(2, 2)))

feature_extractor.add(Flatten())

In [ ]:
x=feature_extractor.output
x=Dense(64,activation='relu')(x)
prediction_layer=Dense(1,activation='sigmoid')(x)

In [ ]:
cnn_model=Model(inputs=feature_extractor.input,outputs=prediction_layer)
cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(cnn_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 256)     7168      
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 254, 256)     1024      
_________________________________________________________________
activation (Activation)      (None, 254, 254, 256)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 256)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 256)     590080    
_________________________________________________________________
activation_1 (Activation)    (None, 125, 125, 256)     0     

In [ ]:
history=cnn_model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_test,y_test))

Epoch 1/3
250/250 [==============================] - 178s 574ms/step - loss: 12.5443 - accuracy: 0.9937 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/3
250/250 [==============================] - 139s 557ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/3
250/250 [==============================] - 139s 557ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [ ]:
feature_extractor.save('/content/drive/MyDrive/Ball_bearing_test_0/Model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Ball_bearing_test_0/Model/assets


Plotting training and validation loss plots

In [ ]:
X_train.shape

(7999, 256, 256, 3)

In [ ]:
# model.save('path/to/location')

Evaluating Test accuracy for Deep Forest

In [ ]:
X_cascade_forest=X_train.reshape(X_train.shape[0],-1)

In [ ]:
model = CascadeForestClassifier(random_state=1)
model.fit(X_cascade_forest, y_train)

In [ ]:
X_test_cascade=X_test.reshape(X_train.shape[0],-1)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))

[2021-07-16 15:35:56.296] Start to fit the model:
[2021-07-16 15:35:56.296] Fitting cascade layer = 0 
[2021-07-16 15:35:57.177] layer = 0  | Val Acc = 97.996 % | Elapsed = 0.880 s
[2021-07-16 15:35:57.182] Fitting cascade layer = 1 
[2021-07-16 15:35:58.090] layer = 1  | Val Acc = 98.144 % | Elapsed = 0.907 s
[2021-07-16 15:35:58.094] Fitting cascade layer = 2 
[2021-07-16 15:35:58.888] layer = 2  | Val Acc = 97.921 % | Elapsed = 0.794 s
[2021-07-16 15:35:58.888] Early stopping counter: 1 out of 2
[2021-07-16 15:35:58.893] Fitting cascade layer = 3 
[2021-07-16 15:35:59.712] layer = 3  | Val Acc = 97.476 % | Elapsed = 0.819 s
[2021-07-16 15:35:59.712] Early stopping counter: 2 out of 2
[2021-07-16 15:35:59.712] Handling early stopping
[2021-07-16 15:35:59.713] The optimal number of layers: 2
[2021-07-16 15:35:59.715] Start to evalute the model:
[2021-07-16 15:35:59.718] Evaluating cascade layer = 0 
[2021-07-16 15:35:59.760] Evaluating cascade layer = 1 

Testing Accuracy: 98.667 %


In [ ]:
X_new, y_new = load_digits(return_X_y=True)
X_new.shape,y_new.shape

((1797, 64), (1797,))

In [ ]:
X.shape,y.shape

((11428, 256, 256, 3), (11428,))

Combining both models to improve accuracy


In [ ]:
X_cascade_forest=feature_extractor.predict(X_train)

ResourceExhaustedError: ignored

In [ ]:
hybrid_model= CascadeForestClassifier(random_state=1)
hybrid_model.fit(X_cascade_forest, y_train)
y_pred = ball_bearing.predict(X_test)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))

In [ ]:
X_1=feature_extractor.predict(X_train[0:100])
X_2= feature_extractor.predict(X_test[10:500])

In [ ]:
from sklearn.ensemble import RandomForestClassifier as rf
rf_model=rf(n_estimators=50,random_state=42)
rf_model.fit(X_1,y_train[0:10])
y_pred = rf_model.predict(X_2)
acc = accuracy_score(y_test[10:50], y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))



Testing Accuracy: 100.000 %


In [ ]:
hybrid_model= CascadeForestClassifier()
hybrid_model.fit(X_1, y_train[0:10])
y_pred = ball_bearing.predict(X_2)
acc = accuracy_score(y_test[10:50], y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))

[2021-07-16 21:14:20.021] Start to fit the model:
[2021-07-16 21:14:20.022] Fitting cascade layer = 0 


MemoryError: ignored